next(p for p in freeagents for x in p.position if x.split('/') == ['2B'])

# Next steps:
1. with the above infrastructure, find a way to change the == to an 'in' clause
2. import in the data such that positions are a string of available positions e.g. '2B-3B' is eligible at 2nd and 3rd base
3. Adjust DoMove below to match

In [45]:
import pandas as pd
import numpy as np

class DraftState:
    def __init__(self, rosters, turns, freeagents, playerjm=None):
        self.rosters = rosters
        self.freeagents = freeagents
        self.turns = turns
        self.playerJustMoved = playerjm

class MLBPlayer:
    def __init__(self, name, team, position, points):
        self.name = name
        self.team = team
        self.position = str(position).split('/')
        self.points = points
    def __repr__(self):
        return self.name
    # def __repr__(self):
    #     return "|".join([self.name, self.team, self.position, str(self.points)])
    def pop(self, index=-1) :
        return self.data.pop(index)

def GetResult(self, playerjm):
    """ Get the game result from the viewpoint of playerjm.
    """
    if playerjm is None: return 0
    
    pos_wgts = {
        ("C"): [1],
        ("1B"): [1],
        ("2B"): [1],
        ("SS"): [1],
        ('3B'): [1],
        # ("1B", "3B"): [.6, .4],
        # ("2B", "SS"): [.6, .4],
        # ('LF'):[.3],
        # ('CF'):[.3],
        ('OF'):[1],
        ("C", "1B", "2B", "SS", "3B", 'DH'): [1],
        ("SP"): [1],
        ("RP"): [1]
    }
    result = 0
    # map the drafted players to the weights
    for p in self.rosters[playerjm]:
        max_wgt, _, max_pos, old_wgts = max(
            ((wgts[0], -len(lineup_pos), lineup_pos, wgts) for lineup_pos, wgts in pos_wgts.items()
                if p.position in lineup_pos),
            default=(0, 0, (), []))
        if max_wgt > 0:
            result += max_wgt * p.points
            old_wgts.pop(0)
            if not old_wgts:
                pos_wgts.pop(max_pos)
                
    # map the remaining weights to the top three free agents
    for pos, wgts in pos_wgts.items():
        result += np.mean([p.points for p in self.freeagents if p.position in pos][:3]) * sum(wgts)
    return result
DraftState.GetResult = GetResult

def GetMoves(self):
    """ Get all possible moves from this state.
    """
    
    pos_max = {"C": 1, "1B": 2, "2B": 2, "SS": 2, "3B": 2, "OF":6, 'P':9, 'DH':1}
    if len(self.turns) == 0: return []
    roster_positions = np.array([p.position for p in self.rosters[self.turns[0]]], dtype=str)
    # moves = [pos for pos, max_ in pos_max.items() if np.sum(roster_positions == pos) < max_]
    moves = list(pos_max.keys())
    return moves
DraftState.GetMoves = GetMoves

def DoMove(self, move):
    """ Update a state by carrying out the given move.
        Must update playerJustMoved.
    """
    player = next(p for p in self.freeagents if p.position == move)
    self.freeagents.remove(player)
    rosterId = self.turns.pop(0)
    self.rosters[rosterId].append(player)
    self.playerJustMoved = rosterId
    
DraftState.DoMove = DoMove

def Clone(self):
    """ Create a deep clone of this game state.
    """
    rosters = list(map(lambda r: r[:], self.rosters))
    st = DraftState(rosters, self.turns[:], self.freeagents[:],
            self.playerJustMoved)
    return st
DraftState.Clone = Clone

# This is a very simple implementation of the UCT Monte Carlo Tree Search algorithm in Python 2.7.
# The function UCT(rootstate, itermax, verbose = False) is towards the bottom of the code.
# It aims to have the clearest and simplest possible code, and for the sake of clarity, the code
# is orders of magnitude less efficient than it could be made, particularly by using a 
# state.GetRandomMove() or state.DoRandomRollout() function.
# 
# Written by Peter Cowling, Ed Powley, Daniel Whitehouse (University of York, UK) September 2012.
# 
# Licence is granted to freely use and distribute for any sensible/legal purpose so long as this comment
# remains in any distributed code.
# 
# For more information about Monte Carlo Tree Search check out our web site at www.mcts.ai
from math import *
import random
class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        UCTK = 200
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result
def UCT(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
    """
    rootnode = Node(state = rootstate)
    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()
        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)
        # Expand
        if node.untriedMoves != []: # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m,state) # add child and descend tree
        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            state.DoMove(random.choice(state.GetMoves()))
        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode
    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited

In [54]:
mlb_players = pd.read_csv("./R-stuff/players.csv", index_col=0)

freeagents = [MLBPlayer(*p) for p in mlb_players.itertuples(index=False, name=None)]

num_competitors = 10
rosters = [[] for _ in range(num_competitors)] # empty rosters to start with

# Add keepers
# all_keepers = [6, 15, 49, 311, 17, 314, 319, 29, 21, 28, 64, 320,
# 9, 13, 16, 22, 14, 18, 19, 42, 3, 11, 41, 302, 2, 4, 8, 94,
# 23, 25, 36, 46, 1, 5, 10, 12, 0, 7, 300, 301]

all_keepers = ['Pete Alonso', 'Fernando Tatis Jr.', 'Wander Franco', 'Sandy Alcantara',
               'Matt Olson', 'Mookie Betts', 'Max Scherzer', 'Brandon Woodruff',
               'Byron Buxton', 'Bo Bichette', 'Alex Bregman', 'Trea Turner',
               'Rafael Devers', 'Kyle Tucker', 'Kyle Schwarber', 'Bryce Harper',
               'Shohei Ohtani', 'Julio Rodríguez', 'Corey Seager', 'Bobby Witt Jr.',
               'Mike Trout', 'José Ramírez', 'Marcus Semien', 'Corbin Burnes',
               'Vladimir Guerrero Jr.', 'Juan Soto', 'Austin Riley', 'Ronald Acuña Jr.',
               'Nolan Arenado', 'Eloy Jiménez', 'Xander Bogaerts', 'Francisco Lindor',
               'Aaron Judge', 'Freddie Freeman', 'Manny Machado', 'Jose Altuve',
               'Paul Goldschmidt', 'Yordan Alvarez', 'Gerrit Cole', 'Justin Verlander']

# These are the list numbers of the players for each manager
Korn = ['Pete Alonso', 'Fernando Tatis Jr.', 'Wander Franco', 'Sandy Alcantara'] # Alonso, Tatis, Franco, Sandy Alc
Alden = ['Matt Olson', 'Mookie Betts', 'Max Scherzer', 'Brandon Woodruff'] # Olson, Gausman, Darvish, Abreu
Kerry = ['Byron Buxton', 'Bo Bichette', 'Alex Bregman', 'Trea Turner'] # Turner, Bregman, Buxton, Bichette
Alex = ['Rafael Devers', 'Kyle Tucker', 'Kyle Schwarber', 'Bryce Harper'] # Devers, Tucker, Acuna, Schwarber
Brent = ['Shohei Ohtani', 'Julio Rodríguez', 'Corey Seager', 'Bobby Witt Jr.'] # Ohtani, Rodriguez, Seager
Dakota = ['Mike Trout', 'José Ramírez', 'Marcus Semien', 'Corbin Burnes'] # Trout, JoRam, Semien, Burnes
Jake = ['Vladimir Guerrero Jr.', 'Juan Soto', 'Austin Riley', 'Ronald Acuña Jr.'] # Vlad Jr, Soto, Riley, Harper
John = ['Nolan Arenado', 'Eloy Jiménez', 'Xander Bogaerts', 'Francisco Lindor'] # Arenado, Eloy, Xander, Lindor
Will = ['Aaron Judge', 'Freddie Freeman', 'Manny Machado', 'Jose Altuve'] # Machado, Freeman, Judge, Altuve
Cuyler = ['Paul Goldschmidt', 'Yordan Alvarez', 'Gerrit Cole', 'Justin Verlander'] # Alvarez, Goldy, Verlander, Cole
teams = [Alex, Alden, Kerry, Korn, Brent, Dakota, Jake, John, Will, Cuyler]
rosters[0] = [p for p in freeagents if p.name in Alex]
rosters[1] = [p for p in freeagents if p.name in Alden]
rosters[2] = [p for p in freeagents if p.name in Kerry]
rosters[3] = [p for p in freeagents if p.name in Korn]
rosters[4] = [p for p in freeagents if p.name in Brent]
rosters[5] = [p for p in freeagents if p.name in Dakota]
rosters[6] = [p for p in freeagents if p.name in Jake]
rosters[7] = [p for p in freeagents if p.name in John]
rosters[8] = [p for p in freeagents if p.name in Will]
rosters[9] = [p for p in freeagents if p.name in Cuyler]

# Remove keepers from free agents list

# freeagents = [freeagents[i] for i in range(0, 598) if i not in all_keepers]
freeagents = [p for p in freeagents if p.name not in all_keepers]



In [48]:
num_rounds = 10
turns = []
# generate turns by snake order
for i in range(num_rounds):
    turns += reversed(range(num_competitors)) if i % 2 else range(num_competitors)
    
state = DraftState(rosters, turns, freeagents)
iterations = 1000
while state.GetMoves() != []:
    move = UCT(state, iterations)
    print(move, end=".")
    state.DoMove(move)

StopIteration: 

In [179]:
pd.DataFrame({"Team " + str(i + 1): r for i, r in enumerate(state.rosters)})

ValueError: All arrays must be of the same length

In [273]:
tm1 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[0])])
tm2 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[1])])
tm3 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[2])])
tm4 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[3])])
tm5 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[4])])
tm6 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[5])])
tm7 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[6])])
tm8 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[7])])
tm9 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[8])])
tm10 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[9])])

In [274]:
results = pd.concat([tm1, tm2, tm3, tm4, tm5, tm6, tm7, tm8, tm9, tm10], axis= 1)
results.columns = ['tm1', 'tm2', 'tm3', 'tm4', 'tm5', 'tm6', 'tm7', 'tm8', 'tm9', 'tm10']

In [283]:
freeagents

[Bo Bichette,
 Jose Altuve,
 Bryan Reynolds,
 Vinnie Pasquantino,
 George Springer,
 Teoscar Hernández,
 Carlos Correa,
 Michael Harris II,
 Nathaniel Lowe,
 Rhys Hoskins,
 Taylor Ward,
 C.J. Cron,
 Will Smith,
 Luis Robert Jr.,
 Byron Buxton,
 Willy Adames,
 Francisco Lindor,
 Josh Bell,
 Rowdy Tellez,
 Ozzie Albies,
 Max Muncy,
 Anthony Santander,
 Salvador Perez,
 Kris Bryant,
 Giancarlo Stanton,
 Ty France,
 Dansby Swanson,
 Nick Castellanos,
 Brandon Nimmo,
 Ryan Mountcastle,
 Masataka Yoshida,
 Tyler O'Neill,
 Randy Arozarena,
 Seiya Suzuki,
 Anthony Rizzo,
 Jorge Polanco,
 J.D. Martinez,
 Mitch Haniger,
 Oneil Cruz,
 Yandy Díaz,
 Willson Contreras,
 Hunter Renfroe,
 Alex Verdugo,
 Matt Chapman,
 Josh Naylor,
 Andrew Vaughn,
 Gleyber Torres,
 Alejandro Kirk,
 Jazz Chisholm Jr.,
 Ketel Marte,
 Tim Anderson,
 Alec Bohm,
 Gunnar Henderson,
 Luis Arraez,
 Ian Happ,
 Andrés Giménez,
 Brandon Lowe,
 Brendan Rodgers,
 Adley Rutschman,
 Jeff McNeil,
 Christian Yelich,
 Ryan McMahon,
 Jak

In [275]:
results

,tm1,tm2,tm3,tm4,tm5,tm6,tm7,tm8,tm9,tm10
0,Pete Alonso,Matt Olson,Bryan Reynolds,Rafael Devers,Shohei Ohtani,Mike Trout,Vladimir Guerrero Jr.,Nolan Arenado,Yordan Alvarez,Aaron Judge
1,Fernando Tatis Jr.,Kevin Gausman,Alex Bregman,Manny Machado,Julio Rodríguez,José Ramírez,Juan Soto,Eloy Jiménez,Freddie Freeman,Paul Goldschmidt
2,Wander Franco,Yu Darvish,J.T. Realmuto,Ronald Acuña Jr.,Corey Seager,Marcus Semien,Austin Riley,Xander Bogaerts,Kyle Tucker,Jacob deGrom
3,Sandy Alcantara,José Abreu,Dylan Cease,Kyle Schwarber,Bobby Witt Jr.,Corbin Burnes,Bryce Harper,Christian Walker,Mookie Betts,Gerrit Cole
4,Jose Altuve,Bo Bichette,Trea Turner,Will Smith,Salvador Perez,Carlos Correa,Carlos Rodón,Vinnie Pasquantino,Nathaniel Lowe,Willy Adames
5,Yandy Díaz,Max Muncy,C.J. Cron,Oneil Cruz,Shane McClanahan,Ozzie Albies,Dansby Swanson,Max Scherzer,Francisco Lindor,Rhys Hoskins
6,George Springer,Matt Chapman,Josh Bell,Aaron Nola,Alec Bohm,Rowdy Tellez,Willson Contreras,Justin Verlander,Gunnar Henderson,Ryan McMahon
7,Tyler Glasnow,Teoscar Hernández,Luis Castillo,Ryan Mountcastle,Ty France,Spencer Strider,Edwin Díaz,Jorge Polanco,Shane Bieber,Alejandro Kirk
8,Adley Rutschman,Sean Murphy,Gleyber Torres,Zack Wheeler,Brandon Woodruff,Daulton Varsho,Jazz Chisholm Jr.,Tim Anderson,Ketel Marte,Luis Arraez
9,Michael Harris II,Jeremy Peña,Jose Miranda,Brandon Lowe,Andrés Giménez,Framber Valdez,Amed Rosario,Nelson Cruz,Robbie Ray,J.D. Martinez
